In [1]:
import tkinter as tk
import tkinter.messagebox
import re
from publish_subscribe import get_poems
import ipynb_importer
from publish_subscribe import Reader
# from tfidf2 import get_sim_dict
# from tfidf2 import get_word2file
# from tfidf2 import get_author2poemid
import pandas as pd
import math
import numpy as np
# from scipy.sparse import csr_matrix
import math

In [6]:
class MainPanel:
    def __init__(self, reader, publisher):
        self.reader = reader
        self.publisher = publisher
        self.poem_dist = get_poems()
        self.all_authors =  list(self.poem_dist.keys())
        self.cur_author = ""
        self.subscribe=[]
        self.sub_dic={}
        self.sim_dit=np.load('sim_char.npy',allow_pickle=True).item()
        self.id2poem=np.load('id2poem.npy',allow_pickle=True).item()
        self.word2file=np.load('word_file_list.npy',allow_pickle=True)
        self.author2poemid=np.load('author2poemid.npy',allow_pickle=True).item()
        self.word2pos=np.load('word2pos.npy',allow_pickle=True).item()
        self.poem_id2pos=np.load('poem_id2pos.npy',allow_pickle=True).item()
        self.cnt_more_10=set(self.sim_dit.keys())
        self.tf_w2f=np.load('tf_w2f.npy',allow_pickle=True)

    
    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.geometry('400x400')
        self.root.title("古诗订阅")
        self.root.iconbitmap('cat.ico')
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.search_button = tk.Button(self.root, text="检索", font=(None,12), command=self.search).pack(pady=40)
        self.root.mainloop()
        
    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.iconbitmap('cat.ico')
        present_author_label = tk.Label(self.add_panel, text=f"订阅列表：{self.subscribe}", font=(None,10)).pack(pady=20)
        label = tk.Label(self.add_panel, text="新增订阅:", font=(None, 10)).pack(pady=20)
#         label_au = tk.Label(self.add_panel, text="作者:").pack(pady=20)
        tk.Label(self.add_panel,text='作者:').place(x=50,y=150)
        tk.Label(self.add_panel,text='关键字:').place(x=50,y=190)
        self.new_author_entry=tk.Entry(self.add_panel)
        self.new_author_entry.place(x=160,y=150)
        self.new_words_entry=tk.Entry(self.add_panel)
        self.new_words_entry.place(x=160,y=190)
#         self.new_author_entry = tk.Entry(self.add_panel)
#         self.new_author_entry.pack()
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.check_new_author).place(x=170,y=240)
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
    
    def check_new_author(self):
        """
        用户点击"确认"按钮后，检查输入是否合法
        
        """
        new_author = self.new_author_entry.get()
        new_words =self.new_words_entry.get()
        if new_words=='':            
            if new_author not in self.all_authors:
                self.hint_label.config(text="目前无此诗人")       
            elif new_author in self.subscribe:
                self.hint_label.config(text="该作者已订阅，请重新输入")
            else:
                tmp_tk = tk.Tk()
                tmp_tk.geometry("300x200")
                tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
                tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
#                 self.reader.subscribeToPublisher(new_author, self.publisher)
                self.subscribe.append(new_author)
                self.sub_dic[new_author]=(1,0)
                self.add_panel.destroy()
                self.update_main_panel()
        else:
            if len(new_words.split())>3:
                self.hint_label.config(text="最多输入3个关键字，关键字用空格隔开")
            else:
                if new_author=='':
                    tempstr=' '.join(new_words.split())
                    self.subscribe.append(tempstr)
                    self.sub_dic[tempstr]=(0,1)
                    tmp_tk = tk.Tk()
                    tmp_tk.geometry("300x200")
                    tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
                    tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
    #                 self.reader.subscribeToPublisher(new_author, self.publisher)
                    self.add_panel.destroy()
                    self.update_main_panel()

                elif new_author not in self.all_authors:
                    self.hint_label.config(text="目前无此诗人")
                else:
                    tempstr=new_author+' '+' '.join(new_words.split())
                    if tempstr in self.subscribe:
                        self.hint_label.config(text="请勿重复订阅！")
                    else:
                        self.subscribe.append(tempstr)
                        self.sub_dic[tempstr]=(1,1)
                        tmp_tk = tk.Tk()
                        tmp_tk.geometry("300x200")
                        tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
                        tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
        #                 self.reader.subscribeToPublisher(new_author, self.publisher)
                        self.add_panel.destroy()
                        self.update_main_panel()

            
    def update_main_panel(self):
        """
        用户新增订阅后，更新主界面
        """
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        it=0
        for sub in self.subscribe:
            it+=1
            button = tk.Button(self.root, text=f"订阅{it}:{sub}", width=25, font=(None,12))
            button.bind("<Button-1>", func=self.handler_adaptor(self.show_all, subscribe = sub))
            button.pack(pady=10)

        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.search_button = tk.Button(self.root, text="检索", font=(None,12), command=self.search).pack(pady=40)
        self.root.update()

    def handler_adaptor(self, fun,  **kwds):
        return lambda event, fun=fun, kwds=kwds: fun(event, **kwds)

    
    def highlight(self,text_widget, keyword, tag):
        pos = '1.0'
        while True:
            idx = text_widget.search(keyword, pos, tk.END)
            if not idx:
                break
            pos = '{}+{}c'.format(idx, len(keyword))
            text_widget.tag_add(tag, idx, pos)
    
    def get_poems_list(self,subscribe=''):

        if self.sub_dic[subscribe]==(1,0):
            self.sim_words=set()
            return self.poem_dist[subscribe],len(self.poem_dist[subscribe])
        elif self.sub_dic[subscribe]==(1,1):
            
            splt=subscribe.split()
            au=splt[0]
            keywords=splt[1:]
            poems_au=self.author2poemid[au]
#             poems_pos=[]
#             for i in poems_au:
#                 poems_pos.append(self.poem_id2pos[i])#得到了这些诗在矩阵中的位置
#             print(poems_au)
            self.sim_words=set()
            score_part1=dict()
            key_poems=[]


            for word in keywords:
                key_poems+=self.word2file[self.word2pos[word]]
                i=0
                for simword,s in self.sim_dit[word]:
                    i+=1
                    self.sim_words.add(simword)
                    if(i==5):
                        break
#             print(sim_words)

            key_poems=list(set(key_poems))
            outpoem_part1=list(set(poems_au).intersection(set(key_poems)))
#             print(key_poems)
            
            sim_poem=[]
            for word in self.sim_words:
                sim_poem+=self.word2file[self.word2pos[word]]
            
            for kp in outpoem_part1:
                score_part1[kp]=0
                for word in keywords:
                    score_part1[kp]+=self.tf_w2f[self.word2pos[word]][self.poem_id2pos[kp]]
#                     score_part1[kp]+=1

            sim_poem=list(set(sim_poem).intersection(set(poems_au)))
            score_part2=dict()            
            for kp in sim_poem:
                if kp not in set(outpoem_part1):
                    score_part2[kp]=0
                    for word in self.sim_words:
                        score_part2[kp]+=self.tf_w2f[self.word2pos[word]][self.poem_id2pos[kp]]
#                         score_part2[kp]+=1

                
            
            p1=sorted(score_part1.items(), key=lambda x: x[1], reverse = True)
            p2=sorted(score_part2.items(), key=lambda x: x[1], reverse = True)

            out_poems=[]
            out1=[]
            out2=[]
#             print(p1)
#             print(p2)
            for k1,k2 in p1:
                out1.append(k1)
            for k1,k2 in p2:
                out2.append(k1)
            out=out1+out2
#             print(out)

            for o in out:
                try:
                    out_poems.append(self.id2poem[o])
                except:print(o)
            return out_poems,len(out_poems)
#             print(out_poems)

            

            
            
        else:
            keywords=subscribe.split()
            self.sim_words=set()
            score_part1=dict()
            key_poems=[]
            self.sim_words=set()

            for word in keywords:
                key_poems+=self.word2file[self.word2pos[word]]
                i=0
                for simword,s in self.sim_dit[word]:
                    i+=1
                    self.sim_words.add(simword)
                    if(i==5):
                        break
#             print(sim_words)

            key_poems=list(set(key_poems))
#             print(key_poems)
            
            sim_poem=[]
            for word in self.sim_words:
                sim_poem+=self.word2file[self.word2pos[word]]
            score_part1=dict()
            for kp in key_poems:
                score_part1[kp]=0
                for word in keywords:
                    score_part1[kp]+=self.tf_w2f[self.word2pos[word]][self.poem_id2pos[kp]]
                    score_part1[kp]+=1


            score_part2=dict()            
            for kp in sim_poem:
                if kp not in set(key_poems):
                    score_part2[kp]=0
                    for word in self.sim_words:
                        score_part2[kp]+=self.tf_w2f[self.word2pos[word]][self.poem_id2pos[kp]]
                        score_part2[kp]+=1

                
            
            p1=sorted(score_part1.items(), key=lambda x: x[1], reverse = True)
            p2=sorted(score_part2.items(), key=lambda x: x[1], reverse = True)

            out_poems=[]
            out1=[]
            out2=[]
#             print(p1)
#             print(p2)
            for k1,k2 in p1:
                out1.append(k1)
            for k1,k2 in p2:
                out2.append(k1)
            out=out1+out2
#             print(out)

            for o in out:
                try:
                    out_poems.append(self.id2poem[o])
                except:print(o)
            return out_poems,len(out_poems)
        
        
    
    def show_all(self, event, subscribe = ''):
        
        self.add_panel = tk.Tk()
        self.add_panel.geometry('500x400')
        self.add_panel.iconbitmap('cat.ico')
        self.now_sub=subscribe
        self.now_poems,self.now_poem_cnt=self.get_poems_list(subscribe)
#         self.now_poems=self.poem_dist['李白']
#         self.now_poem_cnt=len(self.now_poems)
        
        page_str="第1页"
        tk.Label(self.add_panel,text=page_str).pack() 
        tk.Label(self.add_panel,text=f'一共有{math.ceil(self.now_poem_cnt/2)}页').pack()
        tk.Label(self.add_panel,text='请输入页数:').pack()
            
        self.page_entry=tk.Entry(self.add_panel)
        self.page_entry.pack()
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.change_page).pack(pady=20)
        poem1_pos=0
        poem2_pos=1
        gettext=''

        if self.now_poem_cnt>0:
            gettext = '\n'.join(list(self.now_poems[poem1_pos]))
            gettext+='\n\n\n'
        if self.now_poem_cnt>1:
            gettext+='\n'.join(list(self.now_poems[poem2_pos]))

#         poem = tk.Label(self.add_panel, text=gettext, font=(None,10)).pack(pady=20)
        poem= tk.Text(self.add_panel)
        poem.insert(tk.END,gettext)
        keywords=subscribe.split()
#         print(keywords)
        poem.tag_config('tag', background='yellow')
        poem.pack()
        if self.sub_dic[subscribe][0]==1:
            keywords=keywords[1:]
        for word in keywords:
            self.highlight(poem, word, 'tag')
        for word in self.sim_words:
            self.highlight(poem, word, 'tag')

            
    def change_page(self):
        page=int(self.page_entry.get())
        if page>=1 and page<=math.ceil(self.now_poem_cnt/2):
            self.add_panel.destroy()
            self.add_panel = tk.Tk()
            self.add_panel.geometry('500x400')
            self.add_panel.iconbitmap('cat.ico')
            
            page_str="第"+str(page)+"页"
            poem1_pos=2*page-2
            poem2_pos=2*page-1
            gettext=''
            if self.now_poem_cnt>poem1_pos:
                gettext = '\n'.join(self.now_poems[poem1_pos])
                gettext+='\n\n\n'
            if self.now_poem_cnt>poem2_pos:
                gettext+='\n'.join(self.now_poems[poem2_pos])
            tk.Label(self.add_panel,text=page_str).pack() 
            tk.Label(self.add_panel,text=f'一共有{math.ceil(self.now_poem_cnt/2)}页').pack()
            tk.Label(self.add_panel,text='请输入页数:').pack()                        
            self.page_entry=tk.Entry(self.add_panel)
            self.page_entry.pack()   
            confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.change_page).pack(pady=20)
#             poem = tk.Label(self.add_panel, text=gettext, font=(None,10)).pack(pady=20)
            poem= tk.Text(self.add_panel)
            poem.insert(tk.END,gettext)
            subscribe=self.now_sub
            keywords=subscribe.split()
#             print(keywords)
            poem.tag_config('tag', background='yellow')
            poem.pack()
            if self.sub_dic[subscribe][0]==1:
                keywords=keywords[1:]
            for word in keywords:
                self.highlight(poem, word, 'tag')
            for word in self.sim_words:
                self.highlight(poem, word, 'tag')
                
           
            
        else:
            tk.messagebox.showwarning(title='error',message=f'请输入1-{math.ceil(self.now_poem_cnt/2)}中的数字')
        
#     def updata_poems(self):
        

    def show_msg(self, event, author, lb):
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.iconbitmap('cat.ico')
        title = lb.get(lb.curselection()[0])
        title_label = tk.Label(self.add_panel, text=title, font=(None,10)).pack(pady=20)
        for i in self.poem_dist[author]:
            if i[0] == title:
                gettext = '\n'.join(list(i[2:]))
                poem = tk.Label(self.add_panel, text=gettext, font=(None,10)).pack(pady=20)
                break
    
    def search(self):
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.iconbitmap('cat.ico')
        input_label = tk.Label(self.add_panel, text="请输入文字", font=(None,10)).pack(pady=20)
        self.input_entry = tk.Entry(self.add_panel)
        self.input_entry.pack()
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.search_poem).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()

    def search_poem(self):
        input_text, resstr = self.input_entry.get(), []
        for author in self.poem_dist:
            for item in self.poem_dist[author]:
                for poem_item in item:
                    if input_text in poem_item:
                        resstr.append('\n'.join(list(item)))
        if len(resstr) > 0:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("400x300")
            tk.Label(tmp_tk, text = '搜索'+input_text, font=(None, 10)).pack(pady=20)
            bar = tk.Scrollbar(tmp_tk)
            lb = tk.Listbox(tmp_tk, yscrollcommand=bar.set)
            for res in resstr:
                lb.insert('end', res)
            lb.bind("<<ListboxSelect>>", self.handler_adaptor(self.show_info, lb = lb))
            lb.pack(side="bottom", fill="both")
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
            self.add_panel.destroy()
        else:               
            self.hint_label.config(text="无结果")
    
    def show_info(self, event, lb):
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.iconbitmap('cat.ico')
        text = lb.get(lb.curselection()[0])
        title_label = tk.Label(self.add_panel, text=text, font=(None,10)).pack(pady=20)
        
        
